In [1]:
import os
import ast
import time
import json
import pickle
import pandas as pd

import openai

from utils import (
    read_social_listening_data,
    clean_text,
)

from credentials import OPENAI_API_KEY
from codification import Codificacion

In [2]:
# Configurar parámetros
topics_batch_size = 30
sentiment_batch_size = 50
num_topicos = 3
column_name = "text"
id_column = "ID"
cliente = "sostenibilidad"

In [3]:
# Lectura de los datos
data = read_social_listening_data(cliente)
print("Número de filas inicial:", len(data))
data.drop_duplicates([column_name], inplace=True)
print("Número de filas después de eliminar duplicados:", len(data))

Ruta destino...../data/sostenibilidad
Número de filas inicial: 776
Número de filas después de eliminar duplicados: 737


In [4]:
# Definir ID
if id_column in data.columns:
    print(f"La columna ID {(id_column)} ya existe")
else:
    print("Se crea una columna ID")
    data[id_column] = range(0, len(data))

data.head()

Se crea una columna ID


,text,ID
0,IMPULSADORA,0
1,Amigable con el medio ambiente,1
2,Aporte al impuesto del carbono,2
3,equidad,3
4,Menos plástico; ciclo cerrado de producción de...,4


In [5]:
# Limpiar el texto
data[column_name] = clean_text(data, column_name)

In [6]:
# Crear instancia de la clase Codificacion
codificacion = Codificacion(openai_api_key=OPENAI_API_KEY)

In [21]:
# Obtener los tópicos para cada frase
topics = codificacion.get_topics(
    data, batch_size=25, column_name="text", id_column="ID", num_topicos=3
)

# Obtener el sentimiento para cada frase
sentiment = codificacion.get_sentiment(
    data, batch_size=50, column_name="text", id_column="ID"
)

# Obtener la traducción para cada frase
translation = codificacion.get_translation(
    data, batch_size=50, column_name="text", id_column="ID"
)

# Obtener la correción ortográfica para cada frase
spelling_correction = codificacion.get_spelling_correction(
    data, batch_size=50, column_name="text", id_column="ID"
)

Iteración: 0
Iteración: 100
Iteración: 200
Iteración: 300
Iteración: 400
Iteración: 500
Iteración: 600
Iteración: 700


In [7]:
# Obtener el sentimiento para cada frase
sentiment = codificacion.get_sentiment(
    data, batch_size=50, column_name="text", id_column="ID"
)

Iteración: 0
Iteración: 100
Iteración: 200
Iteración: 300
Iteración: 400
Iteración: 500
Iteración: 600
Iteración: 700


In [17]:
import pickle

file_path = "sentiment.pkl"

with open(file_path, "wb") as file:
    pickle.dump(sentiment, file)

print("List saved as a pickle file.")

List saved as a pickle file.


In [13]:
# Convertir la lista de JSON a dataframe
df_topics = codificacion.from_json_list_to_df(topics)
df_sentiment = codificacion.from_list_to_df(sentiment)
df_translation = codificacion.from_json_list_to_df(translation)
df_spelling_correction = codificacion.from_json_list_to_df(spelling_correction)

In [18]:
# Guardar los resultados en un archivo CSV
codificacion.save_pandas_object(df_topics, "../artifacts", cliente, "topics.csv")
codificacion.save_pandas_object(df_sentiment, "../artifacts", cliente, "sentiment.csv")
codificacion.save_pandas_object(
    df_translation, "../artifacts", cliente, "tranlation.csv"
)
codificacion.save_pandas_object(
    df_spelling_correction, "../artifacts", cliente, "spelling_correction.csv"
)

Root directory already exists at ../artifacts\sostenibilidad
